# Signal Sources

In [1]:
import os

# this isn't required with library pip installed
os.chdir('../')


The first building block of Genki Signals is the `SignalSource`

SignalSource is a callable that returns the current value of that Signal. It can be a function or a class with a `__call__` method. One example is the `MouseSignalSource` that gives the current position of the pointer:

In [11]:
from genki_signals.signal_sources import MouseSignalSource

source = MouseSignalSource()
source()

array([1511.98046875,  981.98046875])

Obviously we want to gather multiple samples of this signal. For that we need a `Sampler`.

A Sampler simply samples one or many SignalSources at a given _sample rate_:

In [14]:
from genki_signals.signal_sources import Sampler

# sources is a dictionary mapping names to signal esources
sampler = Sampler(sources = {"mouse": source}, sample_rate=100)

# start gathering samples
sampler.start()

Now the sampler samples the mouse position at a rate of 100 samples / second (hz) (This happens in a separate thread so the cell still returns and the main thread is unblocked.)

We might want to do some processing on these samples. 

This is where SignalFunctions come in. SignalFunctions are functions that take in one or more signals and return another signal.

There is a collection of SignalFunctions available in `genki_signals.signal_functions` and they all have a similar structure, to create one you need to specify some _input names_ and also a name for the output signal: 

In [15]:
import genki_signals.signal_functions as sf

# diff differentiates the "mouse" signals with regard to the "timestamp" and returns the signal "mouse_vel"
# the "mouse" signal is the one we created and "timestamp" is created automatically by the sampler
diff = sf.Differentiate(input_a="mouse", input_b="timestamp", name="mouse_vel")

We need to introduce one more concept to connect all of this together: the SignalSystem.

SignalSystem takes in a Sampler/SignalSource and a list of SignalFunctions

In [17]:
from genki_signals.signal_system import SignalSystem

system = SignalSystem(sampler, [diff])

system.start()

This is the minimal setup we need. With a system, we can add signal functions to do all kinds of processing, and we can do data recording to start building a dataset. However, probably the most useful part of Genki Signals is the real-time visualization. For that we need a frontend: 

# Hard to explain well ?

SignalSystem also has a parameter called update_rate which defines how often we fetch samples from our Sampler.

We can then forward these samples to other places in our program, e.g. a buffer, a plotting class or saving to a file.

This is usefull since batched processing (saving a file / updating plot) is more efficient

In [17]:
# This is how we can stream data into a buffer

from genki_signals.buffers import DataBuffer

buffer = DataBuffer()

system.register_data_feed(id(buffer), lambda data: buffer.extend(data))

In [89]:
buffer

DataBuffer(max_size=None, data=timestamp: (18818,)
mouse: (2, 18820)
mouse_vel: (2, 18879))

Now everytime we call read we get the mouse_positions we have seen before and the mouse velocity at that time.

Now we might want to visualize our Signals to make sure they are how they are supposed to be or to see how they behave in different circumstances.

# TODO: continue with visualization
For that we can use ...

To train a model on these samples it is good practice to store them somewhere.

That brings us to recording.

# What if we don't call read(). then we try to save all samples at once (when stop_recording is called)
We just call system.start_recording(folder_path) which stores the signal samples in a file everytime system.read() is called.

In [39]:
system.start_recording("data/introduction_data/")

FileExistsError: [Errno 17] File exists: 'data/introduction_data'

and system.stop_recording() when we want to stop

In [40]:
system.stop_recording()
system.stop()

AttributeError: 'NoneType' object has no attribute 'stop'

Then we can load this session (stored data + metadata about recording) as follows:

In [11]:
from genki_signals.session import Session

session = Session.from_filename("data/introduction_data/")

session now includes our recorded data as well as metadata about the recording.

# Should Session have a toDataframe() method (data + signalFunctions)

In [23]:
session.data

DataBuffer(max_size=None, data=timestamp: (475,)
mouse: (2, 475))